In [13]:
import pandas as pd

# 데이터 불러오기 

population_df = pd.read_csv('C:/Users/tmdrb/OneDrive//바탕 화면/대학/3학년 2학기/노원구_빅데이터공모전/data/노원구인구수.csv') # 노원구 전체 인구 데이터
population_over_65_df = pd.read_csv('C:/Users/tmdrb/OneDrive/바탕 화면/대학/3학년 2학기/노원구_빅데이터공모전/data/노원구65세이상인구수.csv') # 65세 이상 인구수 데이터
pharmacy_df = pd.read_csv('C:/Users/tmdrb/OneDrive/바탕 화면/대학/3학년 2학기/노원구_빅데이터공모전/data/노원구심야약국후보지.csv') # 약국 후보지 데이터
coords_df = pd.read_excel('C:/Users/tmdrb/OneDrive/바탕 화면/대학/3학년 2학기/노원구_빅데이터공모전/data/노원구행정동위경도.xlsx') # 행정동 위경도 데이터
welfare_recipient_df = pd.read_csv('C:/Users/tmdrb/OneDrive/바탕 화면/대학/3학년 2학기/노원구_빅데이터공모전/data/기초생활수급자_2405.CSV.csv', encoding='cp949') # 기초생활수급자 데이터

# 쉼표 제거 및 숫자로 변환 (필요한 열에 대해서 적용)
population_over_65_df['people_total'] = population_over_65_df['people_total'].str.replace(',', '').astype(float)
population_df['people_total'] = population_df['people_total'].str.replace(',', '').astype(float)


In [12]:
# T점수화 

# Z-score 함수 정의 (표준 정규화)
def z_score(series):
    return (series - series.mean()) / series.std()

# T 점수 계산 함수 정의
def t_score(z):
    return (z * 10) + 50

# 65세 이상 인구수 및 기초생활수급자수 Z-score (표준 정규화)
population_over_65_df['zscore_elderly_population'] = z_score(population_over_65_df['people_total'])
welfare_recipient_df['zscore_welfare_recipient'] = z_score(welfare_recipient_df['기초생계급여수급자'])

# 두 Z-score 값을 0.5씩 곱하고 합산한 후 의료취약지수 계산
population_over_65_df['medical_vulnerability_index'] =  (0.5 * population_over_65_df['zscore_elderly_population']) + (0.5 * welfare_recipient_df['zscore_welfare_recipient'])


# 행정동별 인구수 Z-score 계산
population_df['zscore_total_population'] = z_score(population_df['people_total'])

# 최종적으로 의료취약지수와 Z-score 정규화된 인구수를 곱해 행정동 별 인구에 따른 의료 취약도 점수 산출
population_over_65_df['medical_vulnerable_population'] = t_score(population_over_65_df['medical_vulnerability_index'] * population_df['zscore_total_population'])

# 새로운 결과 데이터 프레임 생성, Z-score와 T 점수 포함
result_data = pd.DataFrame({
    '행정동': population_over_65_df['haengjeongdong'],
    '65세 이상 인구수': population_over_65_df['people_total'],
    '기초생활수급자수': welfare_recipient_df['기초생계급여수급자'],
    '전체인구수' : population_df['people_total'],
    'Z-score 65세 이상 인구수': population_over_65_df['zscore_elderly_population'],
    'Z-score 기초생활수급자수': welfare_recipient_df['zscore_welfare_recipient'],
    'Z-score 전체 인구수': population_df['zscore_total_population'],
    '의료취약지수 ': population_over_65_df['medical_vulnerability_index'],
    '행정동 별 인구에 따른 의료 취약도 점수': population_over_65_df['medical_vulnerable_population']
})

# 결과 데이터프레임 출력
result_data

# # 결과를 CSV 파일로 저장 (한글이 깨지지 않도록 utf-8-sig 인코딩 사용)
# result_data.to_csv('medical_vulnerability_and_population_with_t_scores.csv', index=False, encoding='utf-8-sig')


,행정동,65세 이상 인구수,기초생활수급자수,전체인구수,Z-score 65세 이상 인구수,Z-score 기초생활수급자수,Z-score 전체 인구수,의료취약지수,행정동 별 인구에 따른 의료 취약도 점수
0,월계1동,4489.0,613,20410.0,-0.383999,-0.621757,-0.842452,-0.502878,54.236504
1,월계2동,6428.0,2244,25393.0,0.720278,1.205771,-0.104315,0.963025,48.995417
2,월계3동,6756.0,1568,28525.0,0.907077,0.448316,0.359631,0.677697,52.437206
3,공릉1동,7287.0,1811,35906.0,1.209486,0.720596,1.452986,0.965041,64.021910
4,공릉2동,6418.0,611,41448.0,0.714583,-0.623998,2.273927,0.045293,51.029922
5,하계1동,5033.0,2230,26322.0,-0.074186,1.190084,0.033298,0.557949,50.185787
6,하계2동,3662.0,766,21126.0,-0.854982,-0.450321,-0.736390,-0.652652,54.806063
7,중계본동,2792.0,357,22879.0,-1.350454,-0.908604,-0.476717,-1.129529,55.384653
8,중계1동,2785.0,501,26255.0,-1.354441,-0.747252,0.023374,-1.050847,49.754380
9,중계2.3동,7998.0,1414,19040.0,1.614407,0.275760,-1.045391,0.945083,40.120181
